# 端到端建模流程
## 从原始数据输入到结果文件生成

# 0.引入三方库

In [ ]:
import gc
import numpy as np
import pandas as pd
import lightgbm as lgb

# 1.读取所需数据

In [ ]:
train = pd.read_csv('data/train.csv')
test =  pd.read_csv('data/test.csv')

# 2.做数据预处理

In [ ]:
# 商户信息
def change_object_cols(se):
    value = se.unique().tolist()
    value.sort()
    return se.map(pd.Series(range(len(value)), index=value)).values

In [ ]:
# 减少内存起见，统一对merchant和transaction的id列进行编码
card_id = train['card_id'].values.tolist() + test['card_id'].values.tolist()
card_map = pd.Series(range(len(card_id)), index=card_id)

train['card_id'] = train['card_id'].map(card_map)
test['card_id'] = test['card_id'].map(card_map)
se_map = change_object_cols(train['first_active_month'].append(test['first_active_month']).astype(str))

train['first_active_month'] = se_map[:train.shape[0]]
test['first_active_month'] = se_map[train.shape[0]:]


In [ ]:
df = pd.read_csv('preprocess/baseline_features.csv')

# 4.进行模型训练及预测

In [ ]:
train = pd.merge(train, df, how='left', on='card_id')
test =  pd.merge(test, df, how='left', on='card_id')
del df

In [ ]:
train['card_id'] = train['card_id'].map(pd.Series(card_map.index, index=card_map.values))
test['card_id'] = test['card_id'].map(pd.Series(card_map.index, index=card_map.values))
train.to_csv('preprocess/baseline_train.csv', index=False)
test.to_csv('preprocess/baseline_test.csv', index=False)

In [ ]:
train = pd.read_csv('preprocess/baseline_train.csv')
train.head()

In [ ]:
label = 'target'
features = train.columns.tolist()
features.remove('card_id')
features.remove('target')

train = train.sample(frac=1, random_state=2020)
valid = train[int(0.8*train.shape[0]):].reset_index(drop=True)
train = train[:int(0.8*train.shape[0])].reset_index(drop=True)

In [ ]:
# 读取特征重要性
fse = pd.read_csv('result/baseline_feature_importance.csv', header=None)#.sort_values('1',ascending=False)
fse[1] = features
cols = fse.sort_values(0,ascending=False)[1].values.tolist()

In [ ]:
fse.columns = ['importance', 'feature']
fse[['feature', 'importance']].to_csv('result/baseline_feature_importance.csv', index=False)

In [ ]:
def features_score(num, train_, valid_):
    features = cols[:num]
    # 配置模型的训练参数
    params_initial = {
        'num_leaves':31, 
        'learning_rate':0.1, 
    'boosting':'gbdt',
    'min_child_samples':20,
    'bagging_seed': 2020,
    'bagging_fraction':0.7, 
    'bagging_freq':1,
    'feature_fraction':0.7, 
     'max_depth':-1,
        'metric':'rmse',
    'reg_alpha':0,
    'reg_lambda':1, 
    'objective':'regression'
        }
    ESR = 30
    NBR = 10000
    VBE = 50

    # 转换训练集与验证集为指定的数据格式
    train_ = lgb.Dataset(train_[features], train_[label])
    valid_ = lgb.Dataset(valid_[features], valid_[label])

    # 模型训练
    bst = lgb.train(params_initial,train_, 
          num_boost_round=NBR, valid_sets=[train_,valid_],
          valid_names=['train','valid'],early_stopping_rounds=ESR,verbose_eval=False)

    return bst.best_score['valid']['rmse']
lst = []
for num in range(50, len(cols), 10):
    score = features_score(num, train, valid)
    lst.append(score)
    print(num, score)

In [ ]:
# 配置模型的训练参数
params_initial = {
    'num_leaves':31, 
    'learning_rate':0.1, 
'boosting':'gbdt',
'min_child_samples':20,
'bagging_seed': 2020,
'bagging_fraction':0.7, 
'bagging_freq':1,
'feature_fraction':0.7, 
 'max_depth':-1,
    'metric':'rmse',
'reg_alpha':0,
'reg_lambda':1, 
'objective':'regression'
    }
ESR = 30
NBR = 10000
VBE = 50
features = cols[:140]
# 转换训练集与验证集为指定的数据格式
train = lgb.Dataset(train[features], train[label])
valid = lgb.Dataset(valid[features], valid[label])

In [ ]:
# 模型训练
bst = lgb.train(params_initial,train, 
      num_boost_round=NBR, valid_sets=[train,valid],
      valid_names=['train','valid'],early_stopping_rounds=ESR,verbose_eval=VBE)

# 预测结果
prediction = bst.predict(test[features])
# 获取模型的特征重要性
fse = pd.Series(bst.feature_importance(), index=features)


test['target'] = prediction

submission = test[['card_id', 'target']]

submission['card_id'] = pd.read_csv('data/sample_submission.csv')['card_id'].values

submission.to_csv('result/baseline_select_submission.csv', index=False)

In [ ]:
submission.to_csv('result/baseline_select_submission.csv', index=False)

In [2]:
3.68565*0.7+0.3*3.77840

3.7134749999999994

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

train = pd.read_csv('preprocess/baseline_train.csv').sample(frac=1, random_state=2020)[:1000]
test = pd.read_csv('preprocess/baseline_test.csv')[:1000]

label = "target"
featureSelect = pd.read_csv('preprocess/RF_select_feature.csv', header=None)[0].values.tolist()

features = ["card_id"] + featureSelect + [label]

train = train[features].fillna(0)
test = test[features[:-1]].fillna(0)

# Set the parameters by cross-validation
parameter_space = {
    "n_estimators": [10, 15],
    "min_samples_leaf": [2, 4],
}


print("Tuning hyper-parameters for mse")
clf = RandomForestRegressor(
    n_estimators=30,
    criterion="mse",
    max_depth=5,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.,
    max_features="auto",
    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=4,
    random_state=2020,
    verbose=0,
    warm_start=False
)
grid = GridSearchCV(clf, parameter_space, cv=5, scoring="neg_mean_squared_error")

grid.fit(train[featureSelect].values, train[label].values)

print("Best parameters set found on development set:")
print()
print(grid.best_params_)
print()
print("Grid scores on development set:")
print()

means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()
print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()

bclf = grid.best_estimator_
bclf.fit(train[featureSelect].values, train[label].values)
prediction = bclf.predict(test[featureSelect].values)

In [17]:
pd.Series(prediction).describe()

count    1000.000000
mean       -0.105431
std         0.635695
min        -5.066940
25%        -0.326627
50%        -0.058532
75%         0.301967
max         1.390564
dtype: float64

In [12]:
6.5+8.491

14.991

In [1]:
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()
print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
bclf = grid.best_estimator_
bclf.fit(X_train, y_train)
y_true = y_test
y_pred = bclf.predict(X_test)
y_pred_pro = bclf.predict_proba(X_test)
y_scores = pd.DataFrame(y_pred_pro, columns=bclf.classes_.tolist())[1].values
print(classification_report(y_true, y_pred))
auc_value = roc_auc_score(y_true, y_scores

SyntaxError: unexpected EOF while parsing (<ipython-input-1-b5fb5023c39b>, line 71)

(100, 158)

In [24]:
len(pred)

1000

In [25]:
 grid.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=4,
           oob_score=False, random_state=2020, verbose=0, warm_start=False)